# Get protein altering annotation from VEP output

In [1]:
ls '/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/vep_ensembl/v1/output/'

README  my_output.vcf*  my_output.vcf_summary.html  my_output.vcf_warnings.txt


In [2]:
tsv_path = '/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/vep_ensembl/v1/output/my_output.vcf' # note: it's not actually a VCF file

In [3]:
import numpy as np
import pandas as pd

In [4]:
# positions in vcf files are 1-based in "UCSC BED like format" they are 0-based

### VCF

Be careful with ENSEMBL VEP based on VCF input:

>VEP also supports using VCF (Variant Call Format) version 4.0. This is a common format used by the 1000 genomes project, and can be produced as an output format by many variant calling tools.

>Users using VCF should note a peculiarity in the difference between how Ensembl and VCF describe unbalanced variants. For any unbalanced variant (i.e. insertion, deletion or unbalanced substitution), the VCF specification requires that the base immediately before the variant should be included in both the reference and variant alleles. This also affects the reported position i.e. the reported position will be one base before the actual site of the variant.

>In order to parse this correctly, VEP needs to convert such variants into Ensembl-type coordinates, and it does this by removing the additional base and adjusting the coordinates accordingly.

>**This means that if an identifier is not supplied for a variant (in the 3rd column of the VCF), then the identifier constructed and the position reported in VEP's output file will differ from the input.**


This problem can be overcome with the following:
- ensuring each variant has a unique identifier specified in the 3rd column of the VCF
- using VCF format as output (--vcf) - this preserves the formatting of your input coordinates and alleles
- using --minimal and --allele_number (see Complex VCF entries).


In [5]:
# read the VEP output
colnames = ['Uploaded_variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position',  'Amino_acids',  'Codons',  'Existing_variation', 'Extra' ]

df = pd.read_csv(tsv_path, sep='\t', comment='#', header=None, names=colnames)

# note: it's stupid to read the whole dataframe into memory, probably should have used "grep ..." or similar to only get rows with matches to the relevant categories (see below)

df.shape

(79425722, 14)

### LOF categories

We use the same as in Cirulli et. al, however we don't have any frameshift variants because in v1 we filtered out indels.

In [6]:
lof_anno = pd.Series(['stop_lost','start_lost','splice_donor_variant','frameshift_variant','splice_acceptor_variant','stop_gained'])

# we're being really wasteful with memory here, this needed 38G...
# as said above, if we pre-filtered the variants we could skip this step
is_lof = df['Consequence'].str.split(',', expand=True)
is_lof = np.any(is_lof.apply(lambda x: x.isin(lof_anno), axis=0).values, axis=1)

df = df[is_lof]

In [7]:
df.shape

(458259, 14)

In [8]:
df_summary = df.groupby(['Uploaded_variation','Location','Allele','Gene']).size()

In [ ]:
# 'Allele' is the alternative allele

In [9]:
df_summary.head() # we can now save this table and use it with seak !

Uploaded_variation  Location      Allele  Gene           
10:100048833:G:T    10:100048833  T       ENSG00000120054    2
10:100054346:C:T    10:100054346  T       ENSG00000120054    2
10:100057011:A:G    10:100057011  G       ENSG00000120054    2
10:100063726:C:T    10:100063726  T       ENSG00000120054    2
10:100065187:C:T    10:100065187  T       ENSG00000120054    2
dtype: int64

In [11]:
df_summary.shape

(145816,)

In [13]:
df_summary.groupby(['Gene']).size().describe()

count    19274.000000
mean         7.565425
std          7.512683
min          1.000000
25%          3.000000
50%          6.000000
75%         10.000000
max        188.000000
dtype: float64

In [12]:
# we want to double-check that we can find the corresponding variants in the Janggu output, and order them the same way:

In [14]:
janggu_bed_path_minus = '/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/v2/minus/snps.bed.gz'

In [15]:
janggu_bed_path_plus = '/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/v2/plus/snps.bed.gz'

In [21]:
plus_pos = pd.read_csv(janggu_bed_path_plus, sep='\t', names=['chr','start','end','name'], dtype={'chr':str, 'start':int, 'end':int, 'name':str})

In [29]:
ids = plus_pos['name'].str.split('_').apply(lambda x: x[0])

In [30]:
ids

0             1:925843:A:G
1             1:925845:G:C
2             1:925846:G:T
3             1:925847:G:A
4             1:925852:T:C
                ...       
4858932    24:24864569:C:T
4858933    24:24864598:C:T
4858934    24:24864608:T:A
4858935    24:24864609:A:C
4858936    24:24864611:A:G
Name: name, Length: 4858937, dtype: object

In [32]:
minus_pos = pd.read_csv(janggu_bed_path_minus, sep='\t', names=['chr','start','end','name'], dtype={'chr':str, 'start':int, 'end':int, 'name':str})
ids2 = minus_pos['name'].str.split('_').apply(lambda x: x[0])

In [34]:
ids_all =  pd.concat([ids, ids2], ignore_index=True)

In [36]:
ids_all = ids_all.unique()

In [43]:
df_summary = df_summary.reset_index()

In [44]:
isin_check = df_summary['Uploaded_variation'].isin(ids_all)

In [47]:
df_summary[~isin_check] # 550 missing variants

,Uploaded_variation,Location,Allele,Gene,0
930,10:116621319:G:A,10:116621319,A,ENSG00000266200,3
931,10:116624013:C:T,10:116624013,T,ENSG00000266200,3
932,10:116626073:T:C,10:116626073,C,ENSG00000266200,4
933,10:116627875:G:C,10:116627875,C,ENSG00000266200,3
934,10:116627875:G:T,10:116627875,T,ENSG00000266200,3
...,...,...,...,...,...
145160,9:88132964:A:G,9:88132964,G,ENSG00000177910,1
145161,9:88133652:C:T,9:88133652,T,ENSG00000177910,1
145162,9:88134767:C:T,9:88134767,T,ENSG00000177910,1
145591,9:97292677:T:C,9:97292677,C,ENSG00000254633,1


In [51]:
len(df_summary[~isin_check]['Gene'].unique()) # ... coming from 154 genes 

154

In [52]:
bed_file = '/mnt/dsets/reference_genomes/ensembl/Homo_sapiens.GRCh38.genes.bed'

In [57]:
genes = pd.read_csv(bed_file, sep='\t', header=None, names=['chr','start','end','name','score','strand'])

In [59]:
gene_names = genes['name'].apply(lambda x: x.split('_')[0])

In [67]:
df_summary[~isin_check]['Gene'].isin(gene_names).sum() # only 6 of the variants are in genes we ran Janggu for. still weird but ok...

6

In [70]:
var_pos = df_summary['Uploaded_variation'].str.split(':')

In [76]:
var_pos = np.stack(var_pos.values)

In [78]:
var_pos = pd.DataFrame(var_pos, columns=['chr','pos','ref','alt'])

In [91]:
var_pos['pos']=var_pos['pos'].astype(np.int32)
var_pos['chr']=var_pos['chr'].astype(np.int32) # the Y and X where coded as 24 and 23 

In [98]:
sortorder = var_pos.sort_values(['chr','pos','ref','alt']).index

In [99]:
df_summary = df_summary.loc[sortorder]

In [107]:
df_summary.rename(columns={0:'n_affected'}, inplace=True)

In [109]:
import os

In [111]:
df_summary.to_csv('/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/v2/ensembl_vep/lof_filtered.tsv', sep='\t', index=False)

In [ ]:
df_summary = pd.read_csv()

## Code below were some random tests I did to implement a loader for this type of dataframe

In [123]:
import pandas as pd
import numpy as np

In [124]:
from seak.data_loaders import AnnotationLoader

In [125]:
df_summary = pd.read_csv('/mnt/projects/ukbiobank/derived/projects/kernels_VEP/vep_SPB_out/v2/ensembl_vep/lof_filtered.tsv', sep='\t')

In [126]:
df_summary

,Uploaded_variation,Location,Allele,Gene,n_affected
0,1:69293:C:A,1:69293,A,ENSG00000186092,2
1,1:69745:C:T,1:69745,T,ENSG00000186092,2
2,1:69849:G:A,1:69849,A,ENSG00000186092,2
3,1:69866:C:A,1:69866,A,ENSG00000186092,2
4,1:925960:C:T,1:925960,T,ENSG00000187634,11
...,...,...,...,...,...
145811,24:21549016:C:T,Y:21549016,T,ENSG00000242875,1
145812,24:22180882:T:A,Y:22180882,A,ENSG00000169800,2
145813,24:22405443:G:A,Y:22405443,A,ENSG00000226941,2
145814,24:23167257:C:T,Y:23167257,T,ENSG00000188120,5


In [131]:
loc = np.stack(df_summary['Location'].str.split(':'))
chrom = loc[:,0]
pos = loc[:,1].astype(np.int32)

In [142]:
uploaded_variation = df_summary['Uploaded_variation']

In [135]:
dummy_data = np.random.randn(len(loc),3)

In [138]:
gene = df_summary['Gene']

In [139]:
allele = df_summary['Allele']

In [171]:
iddf =  pd.DataFrame(dummy_data, index=pd.MultiIndex.from_arrays([gene, uploaded_variation], names=['gene','vid']), columns=['x1','x2','x3'])

In [174]:
iddf.loc[(slice(None), '1:69745:C:T'),:] # this works!

,,x1,x2,x3
gene,vid,,,
ENSG00000186092,1:69745:C:T,-0.529817,-1.80892,-0.159462


In [264]:
iddf.loc[(slice(None),['1:69745:C:T','1:69849:G:A']),:].values.shape

(2, 3)

In [236]:
iddf.loc[(slice(None), '1:69745:C:T'),:].shape

(1, 3)

In [ ]:
iddf.loc[(slice(None), '1:69745:C:T'),:].shape

In [230]:
iddf.loc[('ENSG00000186092', slice(None)),] # this works!

x1        x2        x3
gene            vid                                      
ENSG00000186092 1:69293:C:A -0.520332  0.120156  0.368345
                1:69745:C:T -0.529817 -1.808920 -0.159462
                1:69849:G:A  0.579482  0.040292 -2.020403
                1:69866:C:A -1.479302 -0.051390  0.486806

In [231]:
iddf.loc[('ENSG00000186092', slice(None)),].values.shape # this works!

(4, 3)

In [265]:
iddf.loc[(['ENSG00000186092','ENSG00000226941'], slice(None)),].values.shape # this works!

(5, 3)

In [269]:
# watch out, can be empty...
iddf.loc[(['ENSG00000186092','ENSG00000226941'],['1:69745:C:T','1:69849:G:A']),].values.shape 

(2, 3)

In [232]:
iddf.loc[('ENSG00000186092','1:69745:C:T'),].shape #boo!

(3,)

In [246]:
iddf.loc['ENSG00000186092'].loc[['1:69745:C:T','1:69849:G:A']].values

array([[-0.52981709, -1.80892002, -0.15946222],
       [ 0.57948154,  0.04029193, -2.02040341]])

In [273]:
iddf.loc[('ENSG00000186092',['1:69745:C:T']),].shape

(1, 3)

In [237]:
iddf.loc[[('ENSG00000186092','1:69745:C:T'),]].shape

(1, 3)

In [271]:
iddf.loc['ENSG00000186092'].loc[['1:69745:C:T']].shape

(1, 3)

In [286]:
# excluding variants:
iddf

x1        x2        x3
gene            vid                                          
ENSG00000186092 1:69293:C:A     -0.520332  0.120156  0.368345
                1:69745:C:T     -0.529817 -1.808920 -0.159462
                1:69849:G:A      0.579482  0.040292 -2.020403
                1:69866:C:A     -1.479302 -0.051390  0.486806
ENSG00000187634 1:925960:C:T    -1.896887  1.633709 -0.241682
...                                   ...       ...       ...
ENSG00000242875 24:21549016:C:T -0.545992 -0.599044 -0.755441
ENSG00000169800 24:22180882:T:A  0.719216 -0.547347 -0.119536
ENSG00000226941 24:22405443:G:A  1.068421  0.428079 -0.656471
ENSG00000188120 24:23167257:C:T  1.444156  1.370327 -0.458872
ENSG00000185894 24:25043944:A:G -0.066001  0.638623 -0.752907

[145816 rows x 3 columns]

In [290]:
?iddf.index.drop

Signature: iddf.index.drop(codes, level=None, errors='raise')
Docstring:
Make new MultiIndex with passed list of codes deleted

Parameters
----------
codes : array-like
    Must be a list of tuples
level : int or level name, default None
errors : str, default 'raise'

Returns
-------
dropped : MultiIndex
File:      ~/miniconda3/envs/seak/lib/python3.8/site-packages/pandas/core/indexes/multi.py
Type:      method


In [296]:
# dropping variants requires reindexing...
iddf.index.drop('1:69745:C:T', level=1)

MultiIndex([('ENSG00000186092',     '1:69293:C:A'),
            ('ENSG00000186092',     '1:69849:G:A'),
            ('ENSG00000186092',     '1:69866:C:A'),
            ('ENSG00000187634',    '1:925960:C:T'),
            ('ENSG00000187634',    '1:930221:C:T'),
            ('ENSG00000187634',    '1:935806:G:A'),
            ('ENSG00000187634',    '1:935897:G:T'),
            ('ENSG00000187634',    '1:939111:C:T'),
            ('ENSG00000187634',    '1:939379:C:T'),
            ('ENSG00000187634',    '1:941170:G:A'),
            ...
            ('ENSG00000280969', '24:20756856:G:C'),
            ('ENSG00000280969', '24:20768877:C:T'),
            ('ENSG00000183146', '24:21383797:C:G'),
            ('ENSG00000234414', '24:21548164:G:C'),
            ('ENSG00000234414', '24:21549016:C:T'),
            ('ENSG00000242875', '24:21549016:C:T'),
            ('ENSG00000169800', '24:22180882:T:A'),
            ('ENSG00000226941', '24:22405443:G:A'),
            ('ENSG00000188120', '24:23167257:C:T

In [344]:
iddf.index.get_level_values('vid')

Index(['1:69293:C:A', '1:69745:C:T', '1:69849:G:A', '1:69866:C:A',
       '1:925960:C:T', '1:930221:C:T', '1:935806:G:A', '1:935897:G:T',
       '1:939111:C:T', '1:939379:C:T',
       ...
       '24:20756856:G:C', '24:20768877:C:T', '24:21383797:C:G',
       '24:21548164:G:C', '24:21549016:C:T', '24:21549016:C:T',
       '24:22180882:T:A', '24:22405443:G:A', '24:23167257:C:T',
       '24:25043944:A:G'],
      dtype='object', name='vid', length=145816)

In [176]:
iddf.index.droplevel('gene')

Index(['1:69293:C:A', '1:69745:C:T', '1:69849:G:A', '1:69866:C:A',
       '1:925960:C:T', '1:930221:C:T', '1:935806:G:A', '1:935897:G:T',
       '1:939111:C:T', '1:939379:C:T',
       ...
       '24:20756856:G:C', '24:20768877:C:T', '24:21383797:C:G',
       '24:21548164:G:C', '24:21549016:C:T', '24:21549016:C:T',
       '24:22180882:T:A', '24:22405443:G:A', '24:23167257:C:T',
       '24:25043944:A:G'],
      dtype='object', name='vid', length=145816)

In [321]:
posdf = pd.DataFrame(uploaded_variation.values, index=pd.MultiIndex.from_arrays([chrom, pd.IntervalIndex.from_arrays(pos-1,pos,closed='left')], names=['chrom','pos']))
posdf.sort_index(level=['chrom','pos'], inplace=True)

In [328]:
posdf.loc['1'][posdf.loc['1'].index.overlaps(pd.Interval(0,10000000,closed='left'))]

,0
pos,
"[69292, 69293)",1:69293:C:A
"[69744, 69745)",1:69745:C:T
"[69848, 69849)",1:69849:G:A
"[69865, 69866)",1:69866:C:A
"[925959, 925960)",1:925960:C:T
...,...
"[9975730, 9975731)",1:9975731:G:A
"[9981070, 9981071)",1:9981071:C:T
"[9982367, 9982368)",1:9982368:G:A


In [189]:
posdf.loc['1'][posdf.loc['1'].index.overlaps(pd.Interval(69292, 69849, closed='left'))]

array([[('ENSG00000186092', '1:69293:C:A')],
       [('ENSG00000186092', '1:69745:C:T')],
       [('ENSG00000186092', '1:69849:G:A')]], dtype=object)

In [190]:
posdf.loc['1'][posdf.loc['1'].index.overlaps(pd.Interval(69292, 69849, closed='left'))].values

array([[('ENSG00000186092', '1:69293:C:A')],
       [('ENSG00000186092', '1:69745:C:T')],
       [('ENSG00000186092', '1:69849:G:A')]], dtype=object)

In [198]:
iddf.loc[posdf.loc['1'][posdf.loc['1'].index.overlaps(pd.Interval(69292, 69849, closed='left'))].values[:,0]]

x1        x2        x3
gene            vid                                      
ENSG00000186092 1:69293:C:A -0.520332  0.120156  0.368345
                1:69745:C:T -0.529817 -1.808920 -0.159462
                1:69849:G:A  0.579482  0.040292 -2.020403

In [306]:
iddf.loc[posdf.loc['1'][posdf.loc['1'].index.overlaps(pd.Interval(69292, 69700, closed='left'))].values[:,0]]

,,x1,x2,x3
gene,vid,,,
ENSG00000186092,1:69293:C:A,-0.520332,0.120156,0.368345


In [350]:
posdf.loc[posdf[0].isin(pd.Series(['1:69293:C:A','1:69745:C:T']))]

0
chrom pos                        
1     [69292, 69293)  1:69293:C:A
      [69744, 69745)  1:69745:C:T

In [335]:
iddf[slice(0,0)]

,,x1,x2,x3
gene,vid,,,
